In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd
from collections import OrderedDict
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona
import shapely
import datetime

In [21]:
"""
AOI Parameters
"""

# bbox
bbox = (-4.860230,47.428380,-0.982056,48.897512)
bbox_poly = shapely.geometry.box(*bbox, ccw=True)


# start-end times
start = datetime.datetime(2018,4,1)
end = datetime.datetime(2018,9,30)

In [77]:
# define tiles
tiles = ['30UUU', '30UVV', '30UVU', '30TVT', '30UWU', '30TWT', '30UXU']


# set up API
user = "sdonike" ####### ## change this!
pw =  "JH9EZFHhfySLRqTvZR8q" ######## ## change this!
api = SentinelAPI(user, pw, api_url='https://scihub.copernicus.eu/dhus', show_progressbars= True,timeout=200)


In [101]:
def get_footprints(tiles,lower_cloud,upper_cloud,producttype,processinglevel):
    # set query params
    query_kwargs = {
        'platformname': 'Sentinel-2',
        #S2B_MSIL2A_20180929T110829_N0208_R137_T30TWT_20180929T155623,
        #'producttype': producttype,
        'filename' : 'S2B_MSIL1C_20211105T112149_N0301_R037_T30UWU_20211108T162544.SAFE',
        #'processinglevel' : processinglevel,
        #'date': ('20180401', '20180930'),
        #'cloudcoverpercentage': (lower_cloud, upper_cloud),
        # 'limit': 999
    } 
    
    #products_ls = []
    footprints = []

    products = OrderedDict()
    for tile in tiles:
        products = OrderedDict()
        kw = query_kwargs.copy()
        kw['tileid'] = tile  
        pp = api.query(**kw)
        products.update(pp)
        #products_ls.append(products)
        print("Min. CC:",lower_cloud,"- Max. CC",upper_cloud,"\tTile:",tile,"\t - Products:",len(products))


        products_df = api.to_dataframe(products)
        #products_df_sorted = products_df.sort_values(['size'],ascending = [False])
        #products_df_sorted = products_df_sorted.head (1)

        #print(products_df_sorted['size'])
        #print(products_df_sorted['title'])

        fp = []
        for i in products_df_sorted["footprint"]:
            fp.append(i)

        #tmp_ls = [products_df_sorted["title"],products_df_sorted["tileid"],products_df_sorted["footprint"],products_df_sorted["datatakesensingstart"]]

        footprints.append(fp)


        #download data
        # api.download_all(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen',max_attempts=2, checksum = True )
        #api.download(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen', checksum = True )
        #print(products_df_sorted)
    # flatten list
    footprints = [item for sublist in footprints for item in sublist]
    footprints_df = pd.DataFrame(footprints,columns=["geometry"])
    #footprints_df
    
    footprints_df['geometry'] = gpd.GeoSeries.from_wkt(footprints_df['geometry'])
    gdf = gpd.GeoDataFrame(footprints_df, geometry='geometry')

    return(gdf)

In [102]:
get_footprints(tiles,0, 50,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_05.shp')
#get_footprints(tiles,0,50,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_10.shp')  
#get_footprints(tiles,10,15,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_15.shp')  
#get_footprints(tiles,15,20,"S2MSI1C","Level-1C").to_file('qgis_files/sen2_tiles/footprints_20.shp')  

Min. CC: 0 - Max. CC 50 	Tile: 30UUU 	 - Products: 0
Min. CC: 0 - Max. CC 50 	Tile: 30UVV 	 - Products: 0
Min. CC: 0 - Max. CC 50 	Tile: 30UVU 	 - Products: 0
Min. CC: 0 - Max. CC 50 	Tile: 30TVT 	 - Products: 0
Min. CC: 0 - Max. CC 50 	Tile: 30UWU 	 - Products: 1
Min. CC: 0 - Max. CC 50 	Tile: 30TWT 	 - Products: 0
Min. CC: 0 - Max. CC 50 	Tile: 30UXU 	 - Products: 0


In [47]:
products

OrderedDict([('712ce4fa-596b-4fac-8368-91c999ac778c',
              {'title': 'S2B_MSIL1C_20210923T110659_N0301_R137_T30UXU_20210923T120609',
               'link': "https://scihub.copernicus.eu/dhus/odata/v1/Products('712ce4fa-596b-4fac-8368-91c999ac778c')/$value",
               'link_alternative': "https://scihub.copernicus.eu/dhus/odata/v1/Products('712ce4fa-596b-4fac-8368-91c999ac778c')/",
               'link_icon': "https://scihub.copernicus.eu/dhus/odata/v1/Products('712ce4fa-596b-4fac-8368-91c999ac778c')/Products('Quicklook')/$value",
               'summary': 'Date: 2021-09-23T11:06:59.024Z, Instrument: MSI, Satellite: Sentinel-2, Size: 782.24 MB',
               'ondemand': 'false',
               'datatakesensingstart': datetime.datetime(2021, 9, 23, 11, 6, 59, 24000),
               'generationdate': datetime.datetime(2021, 9, 23, 12, 6, 9),
               'beginposition': datetime.datetime(2021, 9, 23, 11, 6, 59, 24000),
               'endposition': datetime.datetime(202